# シリアライザとデシリアライザを理解する

SageMakerの推論エンドポイントにリクエストを投げる際に、

SageMaker -> endpointにて

このノートブックでは、LightGBMがインストールされたカスタムコンテナ構築し、SageMaker Trainingジョブで学習後、推論を行います。
カスタムコンテナの挙動を観察し、SageMakerの推論動作について理解を深めます。

ノートブックは20分程度で実行できます。

## SageMakerの仕組み（仮説）

* SageMakerのコントロールプレーン（サーバ）がある。
    * それは、pingを打って各推論エンドポイントが動いてるかヘルスチェックして、把握している。
    
* 推論エンドポイントへは、.predict()(SageMakerSDKの場合) or invoke_endpoint()(boto3の場合）でデータを投げる
    * predictも結局はinvoke_endpoint()している

https://github.com/aws/sagemaker-python-sdk/blob/885423c26ce7288283bbca7d9c1c53c4d0ccf103/src/sagemaker/predictor.py#L123


invoke_endpoint()すると、SageMakerに推論先を聞きに行き、返された宛先のエンドポイントにデータを投げていると予想。
* endpointやvariantを指定しているので、SageMakerに場所を聞く必要があると予想。SageMakerはDNSのような役割をする。
    * これにより、variantsへのロードバランスをSageMakerが行える。（AutoScaleはSageMakerではなく、他の機構が行なっているはず）
* SageMakerから帰ってきた宛先に/invocationを投げる。推論エンドポイントは/invocationに返答する。

invoke_endpoint()の前の、Predictorクラス作成の時に、SerializerとDeserializerを指定している。
つまり、データ投げる前のクライアント側でシリアライズして、エンドポイントに投げる。
エンドポイントからの応答（推論結果）は、シリアルデータでクライアントに返ってくる。
デシリアライズをクライアント側で実施する。

# シリアライズの確認
シリアライズはクライアント側で実行され、シリアライズされたデータは推論エンドポイントにinvokeされます。

In [25]:
import numpy as np

In [26]:
data_nparr = np.array([0.25387,
                       0.0,
                       6.91,
                       0.0,
                       0.4480,
                       5.399,
                       95.3,
                       5.8700,
                       3.0,
                       233.0,
                       17.9,
                       396.90,
                       30.81])

data_str = '0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66'


In [28]:
print(type(data_nparr))
print(data_nparr)
data_nparr

<class 'numpy.ndarray'>
[2.5387e-01 0.0000e+00 6.9100e+00 0.0000e+00 4.4800e-01 5.3990e+00
 9.5300e+01 5.8700e+00 3.0000e+00 2.3300e+02 1.7900e+01 3.9690e+02
 3.0810e+01]


array([2.5387e-01, 0.0000e+00, 6.9100e+00, 0.0000e+00, 4.4800e-01,
       5.3990e+00, 9.5300e+01, 5.8700e+00, 3.0000e+00, 2.3300e+02,
       1.7900e+01, 3.9690e+02, 3.0810e+01])

In [29]:
print(type(data_str))
print(data_str)
data_str

<class 'str'>
0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81
0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66


'0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66'

In [23]:
# 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()
    print(type(payload))
    print(payload)
print('=' * 20)
payload

<class 'str'>
0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81
0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66
3.67367,0.0,18.1,0.0,0.583,6.312,51.9,3.9917,24.0,666.0,20.2,388.62,10.58
0.29819,0.0,6.2,0.0,0.504,7.686,17.0,3.3751,8.0,307.0,17.4,377.51,3.92
8.15174,0.0,18.1,0.0,0.7,5.39,98.9,1.7281,24.0,666.0,20.2,396.9,20.85
6.65492,0.0,18.1,0.0,0.713,6.317,83.0,2.7344,24.0,666.0,20.2,396.9,13.99
0.17171,25.0,5.13,0.0,0.453,5.966,93.4,6.8185,8.0,284.0,19.7,378.08,14.44
5.73116,0.0,18.1,0.0,0.532,7.061,77.0,3.4106,24.0,666.0,20.2,395.28,7.01
3.1636,0.0,18.1,0.0,0.655,5.759,48.2,3.0665,24.0,666.0,20.2,334.4,14.13
11.8123,0.0,18.1,0.0,0.718,6.824,76.5,1.794,24.0,666.0,20.2,48.45,22.74
8.64476,0.0,18.1,0.0,0.693,6.193,92.6,1.7912,24.0,666.0,20.2,396.9,15.17
0.02177,82.5,2.03,0.0,0.415,7.61,15.7,6.27,2.0,348.0,14.7,395.38,3.11
0.13914,0.0,4.05,0.0,0.51,5.572,88.5,2.5961,5.0

'0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66\n3.67367,0.0,18.1,0.0,0.583,6.312,51.9,3.9917,24.0,666.0,20.2,388.62,10.58\n0.29819,0.0,6.2,0.0,0.504,7.686,17.0,3.3751,8.0,307.0,17.4,377.51,3.92\n8.15174,0.0,18.1,0.0,0.7,5.39,98.9,1.7281,24.0,666.0,20.2,396.9,20.85\n6.65492,0.0,18.1,0.0,0.713,6.317,83.0,2.7344,24.0,666.0,20.2,396.9,13.99\n0.17171,25.0,5.13,0.0,0.453,5.966,93.4,6.8185,8.0,284.0,19.7,378.08,14.44\n5.73116,0.0,18.1,0.0,0.532,7.061,77.0,3.4106,24.0,666.0,20.2,395.28,7.01\n3.1636,0.0,18.1,0.0,0.655,5.759,48.2,3.0665,24.0,666.0,20.2,334.4,14.13\n11.8123,0.0,18.1,0.0,0.718,6.824,76.5,1.794,24.0,666.0,20.2,48.45,22.74\n8.64476,0.0,18.1,0.0,0.693,6.193,92.6,1.7912,24.0,666.0,20.2,396.9,15.17\n0.02177,82.5,2.03,0.0,0.415,7.61,15.7,6.27,2.0,348.0,14.7,395.38,3.11\n0.13914,0.0,4.05,0.0,0.51,5.572,88.5,2.5961,5.0

In [31]:
### str型のCSVフォーマットをシリアライズする場合
from sagemaker.serializers import CSVSerializer

serialized = CSVSerializer().serialize(data_str)
print(type(serialized))
print(serialized)
serialized

<class 'str'>
0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81
0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66


'0.25387,0.0,6.91,0.0,0.448,5.399,95.3,5.87,3.0,233.0,17.9,396.9,30.81\n0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05\n4.64689,0.0,18.1,0.0,0.614,6.98,67.6,2.5329,24.0,666.0,20.2,374.68,11.66'

In [ ]:
### str型のCSVフォーマットをシリアライズする場合
from sagemaker.serializers import CSVSerializer

serialized = CSVSerializer().serialize(data_str)
print(type(serialized))
print(serialized)
serialized

In [38]:
from sagemaker.serializers import NumpySerializer

serialized = NumpySerializer().serialize(data_str)
print(type(serialized))
print(serialized)
serialized

<class 'bytes'>
b"\x93NUMPY\x01\x00v\x00{'descr': '<U216', 'fortran_order': False, 'shape': (), }                                                            \n0\x00\x00\x00.\x00\x00\x002\x00\x00\x005\x00\x00\x003\x00\x00\x008\x00\x00\x007\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x006\x00\x00\x00.\x00\x00\x009\x00\x00\x001\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x004\x00\x00\x004\x00\x00\x008\x00\x00\x00,\x00\x00\x005\x00\x00\x00.\x00\x00\x003\x00\x00\x009\x00\x00\x009\x00\x00\x00,\x00\x00\x009\x00\x00\x005\x00\x00\x00.\x00\x00\x003\x00\x00\x00,\x00\x00\x005\x00\x00\x00.\x00\x00\x008\x00\x00\x007\x00\x00\x00,\x00\x00\x003\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x002\x00\x00\x003\x00\x00\x003\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x001\x00\x00\x007\x00\x00\x00.\x00\x00\x009\x00\x00\x00,\x00\x00\x003\x00\x00\x009\x00\x00\x006\x00\x00\x00.\x00\x00\x009\x00\x00\x00,\x00\x00\x003\x00\x00

b"\x93NUMPY\x01\x00v\x00{'descr': '<U216', 'fortran_order': False, 'shape': (), }                                                            \n0\x00\x00\x00.\x00\x00\x002\x00\x00\x005\x00\x00\x003\x00\x00\x008\x00\x00\x007\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x006\x00\x00\x00.\x00\x00\x009\x00\x00\x001\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x000\x00\x00\x00.\x00\x00\x004\x00\x00\x004\x00\x00\x008\x00\x00\x00,\x00\x00\x005\x00\x00\x00.\x00\x00\x003\x00\x00\x009\x00\x00\x009\x00\x00\x00,\x00\x00\x009\x00\x00\x005\x00\x00\x00.\x00\x00\x003\x00\x00\x00,\x00\x00\x005\x00\x00\x00.\x00\x00\x008\x00\x00\x007\x00\x00\x00,\x00\x00\x003\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x002\x00\x00\x003\x00\x00\x003\x00\x00\x00.\x00\x00\x000\x00\x00\x00,\x00\x00\x001\x00\x00\x007\x00\x00\x00.\x00\x00\x009\x00\x00\x00,\x00\x00\x003\x00\x00\x009\x00\x00\x006\x00\x00\x00.\x00\x00\x009\x00\x00\x00,\x00\x00\x003\x00\x00\x000\x00\x00\x0

SageMakerの動き
* 推論エンドポイントは、シリアル化されたデータを受け取る
* SageMakerのコードで、デシリアライズする。
* input_fn実行
* predict_fn実行
* output_fn実行
* データをシリアライズする。
* クライアントに送信
* クライアント側で、デシリアライズする。

# デシリアライズの確認
クライアントは、推論エンドポイントからシリアルデータを受け取りますので、それをクライアント側でデシリアライズします。

LightGBMは推論結果をndarray型で出力するので、ndarray型をシリアライズして、クライアントに渡すことを想定する。

In [43]:
from sagemaker.deserializers import PandasDeserializer

In [44]:
import botocore
import json
from io import BytesIO

In [45]:
# 返却したいオブジェクト
body_json = {
    "aaa": 3,
    "bbb": [
        {
            "ccc": "ddd"
        }
    ]
}

# エンコード。(encode()はデフォルトでutf-8。)
body_encoded = json.dumps(body_json).encode()

# StreamingBodyへ整形する。
body = botocore.response.StreamingBody(BytesIO(body_encoded),len(body_encoded))

In [46]:
#deserialized = PandasDeserializer().deserialize(body, 'text/csv')
deserialized = PandasDeserializer().deserialize(body, 'application/json') ### JSONがdeserializerのインプット


In [47]:
print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

<class 'pandas.core.frame.DataFrame'>
   aaa             bbb
0    3  {'ccc': 'ddd'}


,aaa,bbb
0,3,{'ccc': 'ddd'}


In [52]:
print(body_nparr)

[19.95642073 27.84489184 23.74743743]


In [72]:
from sagemaker.deserializers import NumpyDeserializer

# 返却したいオブジェクト
body_json = {
    "aaa": 3,
    "bbb": [
        {
            "ccc": "ddd"
        }
    ]
}
body_nparr = np.array([
                        19.95642073217597,
                        27.844891841022335,
                        23.747437427003455
                        ])

# エンコード。(encode()はデフォルトでutf-8。)
body_encoded = json.dumps(body_json).encode()
body_encoded2 = body_nparr.tobytes()

# StreamingBodyへ整形する。
body = botocore.response.StreamingBody(BytesIO(body_encoded),len(body_encoded))
body2 = botocore.response.StreamingBody(BytesIO(body_encoded2),len(body_encoded2))

#deserialized = NumpyDeserializer().deserialize(body, 'application/json') ### JSONがdeserializerのインプット
deserialized = NumpyDeserializer().deserialize(body2, 'application/x-npy') ### JSONがdeserializerのインプット

print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

OSError: Failed to interpret file <_io.BytesIO object at 0x7efcaf7732c0> as a pickle

In [64]:
print(type(body_encoded))
print(body_encoded)

print(type(body))
print(body)


print(type(body_nparr))
print(body_nparr)

print(type(body_encoded2))
print(body_encoded2)

print(type(body2))
print(body2)

<class 'bytes'>
b'{"aaa": 3, "bbb": [{"ccc": "ddd"}]}'
<class 'botocore.response.StreamingBody'>
<class 'numpy.ndarray'>
[19.95642073 27.84489184 23.74743743]
<class 'bytes'>
b'\x84\xe95\xfd\xd7\xf43@!\xd9\xe9\xd4J\xd8;@F\xc9(\x0fX\xbf7@'
<class 'botocore.response.StreamingBody'>


# 参考

botocore.response

https://botocore.amazonaws.com/v1/documentation/api/latest/reference/response.html

raw_streamを入力する必要がある。


バイナリ I/O
https://docs.python.org/ja/3/library/io.html#binary-i-o


BytesIO はインメモリーのバイナリストリームです:

f = io.BytesIO(b"some initial binary data: \x00\x01")

In [77]:
from sagemaker.deserializers import NumpyDeserializer


# StreamingBodyへ整形する。
body = botocore.response.StreamingBody(BytesIO(b'{"hogehoge":1}'),len(b'{"hogehoge":1}'))

deserialized = NumpyDeserializer().deserialize(body, 'application/json') ### JSONがdeserializerのインプット
#deserialized = NumpyDeserializer().deserialize(body, 'application/x-npy') ### JSONがdeserializerのインプット

print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

<class 'numpy.ndarray'>
{'hogehoge': 1}


array({'hogehoge': 1}, dtype=object)

In [78]:
from sagemaker.deserializers import NumpyDeserializer

# StreamingBodyへ整形する。
body = botocore.response.StreamingBody(BytesIO(b'{"hogehoge":1}'),len(b'{"hogehoge":1}'))

deserialized = NumpyDeserializer().deserialize(body, 'application/x-npy') ### ndarrayがdeserializerのインプット

print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

OSError: Failed to interpret file <_io.BytesIO object at 0x7efcaf764e00> as a pickle

In [ ]:
body_nparr = np.array([
                        19.95642073217597,
                        27.844891841022335,
                        23.747437427003455
                        ])

In [105]:
# np.load(io.BytesIO(stream.read()), allow_pickle=self.allow_pickle)　が動かないとエラー

np.load(BytesIO(b'[1,1,1]'), allow_pickle='allow_pickle') # np.load()でエラー発生

OSError: Failed to interpret file <_io.BytesIO object at 0x7efcaf473090> as a pickle

In [83]:
body_nparr = np.array([
                        19.95642073217597,
                        27.844891841022335,
                        23.747437427003455
                        ])

In [85]:
body_nparr

array([19.95642073, 27.84489184, 23.74743743])

In [86]:
np.save('hoge', body_nparr)

In [101]:
body = botocore.response.StreamingBody(b'{"hogehoge":1}',len(b'{"hogehoge":1}'))

In [102]:
body.read()

AttributeError: 'bytes' object has no attribute 'read'

In [99]:
body.seek()

UnsupportedOperation: seek

In [90]:
BytesIO(b'{"hogehoge":1}')


nupy.load()

https://numpy.org/doc/stable/reference/generated/numpy.load.html

The file to read. File-like objects must support the seek() and read() methods and must always be opened in binary mode. 

In [116]:
BytesIO(b'[1,1,1]').seek(50000)

50000

In [119]:
BytesIO(b'[1,1,1]').read(10000)

b'[1,1,1]'

BytesIOはseekもreadもできる。

In [132]:
np.load(BytesIO(b' a'), allow_pickle=True) # np.load()でエラー発生

OSError: Failed to interpret file <_io.BytesIO object at 0x7efcaf3def90> as a pickle

b'aaaa'のバイト列がいけてないのか？pickleであることを示す文字列がない？で、seek()で失敗している？？


中身が想定しているものではないのかも
https://teratail.com/questions/302899

ファイルがおかしい場合にエラーとなっている事例のようだ


In [141]:
np.load(BytesIO(body_nparr.dumps()), allow_pickle=True) # np.load()でエラー発生

array([19.95642073, 27.84489184, 23.74743743])

# 解答
numpyのndarrayを、ファイルではなく、pickle文字列に変換する必要がある。
そのために、numpy.ndarray.dumps()を使う

https://numpy.org/doc/stable/reference/generated/numpy.ndarray.dumps.html

In [143]:
np.load(BytesIO(body_nparr.dumps()), allow_pickle=True) # np.load()でエラー発生

### こうすることで、doby_nparrがpickleのstringに変換され、BytesIO()によってseek()もread()もできるストリーム（file_alike)に変換される。
### np.load()でこれを読み込むことができる。

array([19.95642073, 27.84489184, 23.74743743])

In [135]:
body_nparr = np.array([
                        19.95642073217597,
                        27.844891841022335,
                        23.747437427003455
                        ])

In [138]:
body_nparr.tobytes()

b'\x84\xe95\xfd\xd7\xf43@!\xd9\xe9\xd4J\xd8;@F\xc9(\x0fX\xbf7@'

# 0.実行環境確認
本ノートブックは、SageMakerノートブックインスタンス上で動作確認しています。
* インスタンスタイプ：ml.t3.medium
* カーネル：conda_python3

## 0-1.pythonバージョン確認

In [ ]:
#Pythonのバージョン情報
import sys
sys.version # 3.8.12

In [ ]:
# Pythonのバージョン確認 (システムコマンド使用）
!python -V # 3.8.12

## 0-2.SageMakerSDKバージョン確認

Amazon SageMaker Python SDKは、Amazon SageMaker上で機械学習されたモデルをトレーニングおよびデプロイするためのオープンソースライブラリです。

このSDKを使用すると、一般的な深層学習フレームワーク、Amazonが提供するアルゴリズム、またはSageMaker互換のDockerイメージに組み込まれた独自のアルゴリズムを使ってモデルをトレーニングおよびデプロイすることができます。

* ドキュメント : https://sagemaker.readthedocs.io/en/stable/
* GitHub : https://github.com/aws/sagemaker-python-sdk

SageMakerSDK をインポートすると、バケットが作成されます。  
sagemaker-＜region＞-＜account＞

In [ ]:
# SageMakerSDK のバージョン確認
import sagemaker
print('Current SageMaker Python SDK Version ={0}'.format(sagemaker.__version__)) # 2.110.0

# 1.データ準備

学習、推論で利用するデータを準備します。

scikit-learn付属の、ボストン住宅価格データセットを利用します。(注：バージョン1.2から除外されます）  
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html

以下のスクリプトを参考にしています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/blob/main/lightgbm_bring_your_own_container_local_training_and_serving/lightgbm_bring_your_own_container_local_training_and_serving.py

In [2]:
import sklearn
sklearn.__version__ # 1.0.1

'1.0.1'

In [3]:
import pandas as pd
pd.__version__ # 1.3.4

'1.3.4'

## 1-1. データロード

In [4]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

In [5]:
data = load_boston() # 1.2でデータセットがなくすという警告が出ますが動作に影響ありません

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the Calif

## 1-2. 特徴量生成（Feature Engineering）
本ノートブックでは実施しません。そのままデータを利用します。

## 1-3. データ分割
学習用（train）、評価用（validation）、テスト用（test）にデータを分割します。  
train:val:test = 3(60%):1(20%):1(20%)に分割します。  

In [6]:
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.2, random_state=45)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=45)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX['target'] = y_train

valX = pd.DataFrame(X_val, columns=data.feature_names)
valX['target'] = y_val

testX = pd.DataFrame(X_test, columns=data.feature_names)

In [7]:
# 確認
print(trainX.shape)
trainX.head()

(303, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43,22.9
1,0.33983,22.0,5.86,0.0,0.431,6.108,34.9,8.0555,7.0,330.0,19.1,390.18,9.16,24.3
2,0.10469,40.0,6.41,1.0,0.447,7.267,49.0,4.7872,4.0,254.0,17.6,389.25,6.05,33.2
3,6.80117,0.0,18.10,0.0,0.713,6.081,84.4,2.7175,24.0,666.0,20.2,396.90,14.70,20.0
4,1.35472,0.0,8.14,0.0,0.538,6.072,100.0,4.1750,4.0,307.0,21.0,376.73,13.04,14.5


In [8]:
# 確認
print(valX.shape)
valX.head()

(101, 14)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.03150,95.0,1.47,0.0,0.403,6.975,15.3,7.6534,3.0,402.0,17.0,396.90,4.56,34.9
1,0.51183,0.0,6.20,0.0,0.507,7.358,71.6,4.1480,8.0,307.0,17.4,390.07,4.73,31.5
2,19.60910,0.0,18.10,0.0,0.671,7.313,97.9,1.3163,24.0,666.0,20.2,396.90,13.44,15.0
3,0.95577,0.0,8.14,0.0,0.538,6.047,88.8,4.4534,4.0,307.0,21.0,306.38,17.28,14.8
4,0.09604,40.0,6.41,0.0,0.447,6.854,42.8,4.2673,4.0,254.0,17.6,396.90,2.98,32.0


In [9]:
# 確認
print(testX.shape)
testX.head()

(102, 13)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.25387,0.0,6.91,0.0,0.4480,5.399,95.3,5.8700,3.0,233.0,17.9,396.90,30.81
1,0.01951,17.5,1.38,0.0,0.4161,7.104,59.5,9.2229,3.0,216.0,18.6,393.24,8.05
2,4.64689,0.0,18.10,0.0,0.6140,6.980,67.6,2.5329,24.0,666.0,20.2,374.68,11.66
3,3.67367,0.0,18.10,0.0,0.5830,6.312,51.9,3.9917,24.0,666.0,20.2,388.62,10.58
4,0.29819,0.0,6.20,0.0,0.5040,7.686,17.0,3.3751,8.0,307.0,17.4,377.51,3.92


In [10]:
# 確認
y_test[0:5]

array([14.4, 33. , 29.8, 21.2, 46.7])

## 1-4.データ保存
ローカル、S3それぞれにデータを保存します。

### 1-4-1.ローカルへ保存

In [11]:
# ディレクトリ作成
from pathlib import Path

Path('./data/train').mkdir(parents=True, exist_ok=True)
Path('./data/valid').mkdir(parents=True, exist_ok=True)
Path('./data/test').mkdir(parents=True, exist_ok=True)

In [12]:
# ローカルへ保存
local_train = './data/train/boston_train.csv'
local_valid = './data/valid/boston_valid.csv'
local_test = './data/test/boston_test.csv'

trainX.to_csv(local_train, header=None, index=False)
valX.to_csv(local_valid, header=None, index=False)
testX.to_csv(local_test, header=None, index=False)

### 1-4-2.S3へ保存

一意のバケット作成のために、sgemaker.Session().default_bucket()を利用します。

https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session

sagemaker-＜region＞-＜accoutid＞　を取得することができます。

In [13]:
bucket_name = sagemaker.Session().default_bucket()
region_name = sagemaker.Session().boto_region_name
account_id =  sagemaker.Session().account_id()

NameError: name 'sagemaker' is not defined

In [14]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

NameError: name 'bucket_name' is not defined

In [15]:
# バケット作成(SageMakerSDKのインポート時作成されています。他のバケット作成時に利用ください)
#import boto3

#s3_resource = boto3.resource('s3')
#s3_resource.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region_name})

In [16]:
# S3へ保存
train_s3 = sagemaker.s3.S3Uploader.upload('./data/train/boston_train.csv', f's3://{bucket_name}/demo_lightgbm/train')
valid_s3 = sagemaker.s3.S3Uploader.upload('./data/valid/boston_valid.csv', f's3://{bucket_name}/demo_lightgbm/valid')

NameError: name 'sagemaker' is not defined

In [17]:
# 確認:格納したS3のURIが返されています
print(train_s3)
print(valid_s3)

NameError: name 'train_s3' is not defined

# 2.LightGBMカスタムコンテナの構築


学習用のカスタムコンテナの作成には大きく分けて3つのパターンがあります。詳細は以下のブログを参考ください。

https://aws.amazon.com/jp/blogs/news/sagemaker-custom-containers-pattern-training/

まずはSageMakerの動作を理解するためにベースイメージ(ubuntu:16.04) + カスタムレイヤー方式を採用します。

## 2-1. Dockerfileの確認

資材はこちらのノートブックを参考に準備しています。

https://github.com/aws-samples/amazon-sagemaker-local-mode/tree/main/lightgbm_bring_your_own_container_local_training_and_serving/container

まずは、Dockerfileを確認します。

In [ ]:
!pygmentize ./container/Dockerfile

### 解説：推論エンドポイント構築時のSageMakerの動作
SageMakerの推論エンドポイントのデプロイは、SageMaker SDKでは、deploy()メソッドで実行します。

https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html

その際に、SageMakerは以下のコマンドを実行します。

docker run < Docker image > server

今回のカスタムコンテナでは、 /opt/program に配置した serve スクリプトが実行されます。

serveスクリプトを確認してみましょう。

In [ ]:
!pygmentize -l py ./container/lightgbm_regression/serve

末尾の start_server() を実行しており、start_server()では以下が行われます。

* nginxの起動（Webサーバ/リバースプロキシの役割）
    * nginx.confを読み込みます。
* gunicornの起動（Applicationサーバの役割）
    * gunicornの起動コマンド引数に'wsgi:app'とあるように、wsgiモジュールwsgi.pyの、appアプリケーションを読み込みます。

nginx.confを確認します。

In [ ]:
!pygmentize ./container/lightgbm_regression/nginx.conf

SageMakerから受け取った /ping と /invocations リクエストを上記で設定したgunicornに渡します。
以下に記載があるように、ポート8080を利用する必要があります。

https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html

How Containers Serve Requests  
Containers need to implement a web server that responds to /invocations and /ping on port 8080.

次に、gunicornへのアプリケーションのキック用に使われるファイル wsgi.pyを確認します。

predictor.py の、appを読み込んでいることがわかります。

In [ ]:
!pygmentize ./container/lightgbm_regression/wsgi.py

predictor.py を確認します。

flaskフレームワークを用いて、/ping, /invocations に対する処理を実装していることがわかります。

In [ ]:
!pygmentize ./container/lightgbm_regression/predictor.py

## 2-2. dockerイメージの build & push
上記で確認したカスタムコンテナをビルドします。

ビルド&pushには7分ほどかかります。

In [ ]:
%%sh

# The name of our algorithm
algorithm_name=sagemaker-lightgbm-regression

cd container

chmod +x lightgbm_regression/train
chmod +x lightgbm_regression/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to ap-northeast-1 if none defined)
region=$(aws configure get region)
region=${region:-ap-northeast-1}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

## 2-3. 学習前設定
AWSコンソールでECRに移動し、作成したコンテナがあることを確認します。

image URIを設定します。

In [ ]:
# 確認
print(bucket_name)
print(region_name)
print(account_id)

In [ ]:
# imageURLの設定
image_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/sagemaker-lightgbm-regression'

In [ ]:
# 確認
image_uri

In [ ]:
# 学習で指定するLightGBMのハイパーパラメータを設定します。
hyperparameters={'boosting_type': 'gbdt',
            'objective': 'regression',
            'num_leaves': 31,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0}

## 2-4.ローカル学習の実行
まずはローカルモードでモデルの学習を行います。
ローカルモードを利用することで、コンテナイメージのダウンロードや展開の手間を省くことができるため、コードのデバッグを行う場合に便利です。

ECRからビルドしたイメージを持ってきて、ローカルのdockerでビルドして、実行する

In [ ]:
# ローカルファイルのパスを設定（S3パス指定も可）
train_location = 'file://'+local_train
valid_location = 'file://'+local_valid

print(train_location)
print(valid_location)

In [ ]:
from sagemaker.estimator import Estimator

In [ ]:
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
# 確認
role

SageMakerのEstimatorを作成します。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html

In [ ]:
local_lightgbm = Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type="local",
    hyperparameters=hyperparameters
    )

fitメソッドで学習ジョブを発行します

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit

In [ ]:
local_lightgbm.fit({'train':train_location, 'validation': valid_location})

ローカルモードの学習結果についてもS3に保管されます。

s3://sagemaker-< リージョン名 >-< アカウントID >/sagemaker-lightgbm-regression-yyyy-MM-dd-HH-mm-ss-fff/

* model.tar.gz
* output.tar.gz

Trainingジョブの詳細について学びたい場合は、BlackBeltの解説もご参照ください。
https://www.youtube.com/watch?v=byEawTm4O4E

## 2-5.ローカルデプロイ

serializer : インプットデータの形式を指定します。
https://sagemaker.readthedocs.io/en/stable/v2.html

In [ ]:
# 事前準備：全コンテナ停止
!docker stop $(docker ps -q)

In [ ]:
# 確認
!docker ps

起動中のコンテナイメージがないことを確認し、ローカルデプロイを行います。

In [ ]:
local_predictor = local_lightgbm.deploy(1, 'local', serializer=sagemaker.serializers.CSVSerializer()) 

In [ ]:
# 確認
!docker ps

ローカルにコンテナイメージが展開されていることが確認できました。

## 2-6.ローカルエンドポイントで推論実施

In [ ]:
# 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = local_predictor.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

## 2-7.学習ジョブを発行
次は、ローカルモードではなく、
同じカスタムコンテナで、学習ジョブを実行します。

Estimatorの引数instance_typeにインスタンスタイプを指定することで、学習ジョブが発行されます。

In [ ]:
# 確認
print(train_s3)
print(valid_s3)

In [ ]:
est_lightgbm = Estimator(
    image_uri,
    role,
    instance_count=1,
    instance_type="ml.m4.2xlarge", # インスタンスタイプを指定
    hyperparameters=hyperparameters)

In [ ]:
est_lightgbm.fit({'train':train_s3, 'validation': valid_s3})

学習には3分ほど時間がかかります。

課金されるのは75秒ほどです。

## 2-8.エンドポイントにデプロイ

デプロイすると、
SageMaker は docker run <image> serveを実行します。

    
デプロイには3分ほどかかります。

In [ ]:
from sagemaker.predictor import csv_serializer

deployメソッドで、推論エンドポイントをデプロイします。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy

In [ ]:
predictor = est_lightgbm.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer, wait=True)

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor.predict(payload).decode('utf-8')
print(predicted)

# 3.推論コードの外部指定、フロントエンドはSageMakerが準備した仕組みを利用する。
推論コードを外部から指定するために、SageMaker Inference Toolkitを導入します。

https://github.com/aws/sagemaker-inference-toolkit

また、前セクションnginx, gunicorn, flaskを用いて実装したモデルサービングの仕組みはSageMaker側で準備されたものを流用します。
これは、MMS(Multi Model Server)というライブラリを導入します。

https://github.com/awslabs/multi-model-server/tree/master/docker

* SageMaker-Inference-Toolkitと、Multi Model Serverを導入する
* ビルトインコンテナ + requirements.txt, inference.pyを利用する

MMSの利用については、以下のサンプルコードも参照ください。

https://github.com/aws/amazon-sagemaker-examples/tree/main/advanced_functionality/multi_model_bring_your_own


## 3-1.Dockerfileの確認

まずは、利用するDockerfileを確認します。
MMSに必要なJavaをインストールし、MMSとinference-toolkitをインストールしています。

lightgbmはrequirements.txtでインストールを試みるため、Dockerfileには記載していません。（記載することも可能）

In [ ]:
!pygmentize ./container_sminftoolkit/Dockerfile

## 3-2.エントリポイントを確認

SageMakerSDKにてdeploy()を実行した際の

docker run \<image> server

で実行される、ENTRYPOINTを確認します。

dockerfile

これは、以下に該当する。

3.Implement a serving entrypoint, which starts the model server.


https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/model_server.py

start_model_server()は、引数指定しない場合、

DEFAULT_HANDLER_SERVICE = default_handler_service.__name__

を指定。これは、inference-toolkitのハンドラサービスである。

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/default_handler_service.py



ハンドラサービスが、Transformer()を作り、そのなかで、推論ハンドラが作られている。

DefaultHandlerService -> Transformer -> DefaultInferenceHandler

https://github.com/aws/sagemaker-inference-toolkit/blob/3774c1a0fb4408cfa95333b75d6e30a376bffa52/src/sagemaker_inference/transformer.py


In [ ]:
!pygmentize ./container_sminftoolkit/dockerd-entrypoint.py

start_model_server()は引数指定しない場合、
inference-toolkitのTransform()が作られる。

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/model_server.py

DEFAULT_HANDLER_SERVICE = default_handler_service.__name__

より、

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/default_handler_service.py

__init__にて、Trransformer()が実行

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/transformer.py

Transform()において、inference-toolkitのDefaultInferenceHandlerが利用される。

https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/default_inference_handler.py

よって、このdockerd-entrypoint.pyが最小構成となる。

## 解説
ハンドラサービスと推論ハンドラがある。

ハンドラサービスは、以下に該当する。

2.Implement a handler service that is executed by the model server.

モデルの推論ハンドラは、以下に該当する。

1.Implement an inference handler, which is responsible for loading the model and providing input, predict, and output functions. 


2.のハンドラサービスから、1.の推論ハンドラがロードされる。推論ハンドラはinference-toolkitで用意したものを使ってもよい。

In [ ]:
#!pygmentize ./container_sminftoolkit/model_handler.py ### 最小構成には不要

build&pushには3分ほどかかります。

In [ ]:
%%sh

# The name of our algorithm
#algorithm_name=demo-sagemaker-multimodel
algorithm_name=demo-sagemaker-inftoolkit

#cd container
cd container_sminftoolkit

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -q -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

* dockerd-entrypoint.py が実行され、サーバーの起動を試みる。
    * サーバー起動の際に必要はハンドラーは、odel-handler.pyに記載されている。
    


## ローカルにエンドポイントをデプロイ
モデルは前のセクションで作成したLGBMモデル

* ソースも指定する
* LGBMはrequirements.txtでインストールする

In [ ]:
container_uri = f'{account_id}.dkr.ecr.{region_name}.amazonaws.com/demo-sagemaker-inftoolkit:latest'

In [ ]:
container_uri

In [ ]:
### 2.8の学習ジョブで構築したモデルを利用する
#est_lightgbm.model_data

### ローカル学習で構築したモデルを利用する場合
model_data=local_lightgbm.model_data

In [ ]:
!docker ps

In [ ]:
#全コンテナ停止
!docker stop $(docker ps -q)

In [ ]:
!docker ps

In [ ]:
from sagemaker.predictor import RealTimePredictor

lgb_model = sagemaker.model.Model(#est_xgb.image_uri, # XGBoostビルトインコンテナのURI
                                  container_uri,
                                  model_data=model_data, # ローカル学習で生成したモデルファイル
                                  role=role,
                                  predictor_cls=RealTimePredictor, # 推論するための識別子を指定
                                  source_dir='./src_builtin_container_serve', # requirements.txt必要な場合
                                  entry_point='inference.py' # source_dirを指定している場合、.pyファイルを指定する。
                                  #entry_point='./src_builtin_container_serve/inference.py'
                                 )

In [ ]:
predictor_lgb_model = lgb_model.deploy(initial_instance_count=1,
                                       instance_type='local', 
                                       serializer=csv_serializer, ### string形式でSageMakerに渡す（認識してもらう）
                                      )

In [ ]:
!docker ps

In [ ]:
#!docker stop f380dc891702

In [ ]:
#!docker ps

## 推論実施

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor_lgb_model.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

In [ ]:
print(predicted)

In [ ]:
print(type(predicted))
predicted

## (option)返り値をstr以外で受け取りには
Deserializerの説明

現在は、この動画にあるように、SageMakerSDKを使っているため、
deserializerがxxxが使われます。

指定することで、numpy_arrayなどで受け取ることができます。

https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy

deploy()の中で、deserializerを指定します。

In [ ]:
!docker stop $(docker ps -q)

In [ ]:
from sagemaker.deserializers import PandasDeserializer

In [ ]:
#PandasDeserializer.deserialize([1,2,3], “text/csv”,)
PandasDeserializer.deserialize(stream=[1,2,3], content_type='application/json',)

In [ ]:
predictor_lgb_model = lgb_model.deploy(initial_instance_count=1,
                                       instance_type='local', 
                                       serializer=csv_serializer, ### string形式でSageMakerに渡す（認識してもらう）
                                       deserializer=PandasDeserializer
                                      )

https://aws.amazon.com/jp/blogs/aws/amazon-sagemaker-serverless-inference-machine-learning-inference-without-worrying-about-servers/

In [ ]:
input_jsonlines = [
    {"features": ["I love this product!"]},
    {"features": ["OK, but not great."]},
    {"features": ["This is not the right product."]},
]

In [ ]:
from sagemaker.serializers import JSONLinesSerializer

In [ ]:
JSONLinesSerializer().serialize(input_jsonlines)

In [ ]:
### CSVフォーマットをシリアライズする場合

from sagemaker.serializers import CSVSerializer

input_csv = [['a1','a2'],
             ['b1','b2'],
             ['c1','c2']]

serialized = CSVSerializer().serialize(input_csv)
print(type(serialized))
print(serialized)

CSVSerializer().serialize(input_csv)

In [ ]:
### CSVフォーマットをシリアライズする場合

from sagemaker.serializers import CSVSerializer

input_csv = 'a,b,c,d,e'

serialized = CSVSerializer().serialize(input_csv)
print(type(serialized))
print(serialized)

CSVSerializer().serialize(input_csv)

In [ ]:
### CSVフォーマットをシリアライズする場合

from sagemaker.serializers import CSVSerializer

input_csv = [[1,2],
             [3,4],
             [5,6]]

serialized = CSVSerializer().serialize(input_csv)
print(type(serialized))
print(serialized)

CSVSerializer().serialize(input_csv)

In [ ]:
sagemaker.serializers.NumpySerializer

In [ ]:
import numpy as np
np.array([1,2,3,4])


NumpyArrayにシリアライズ
https://sagemaker.readthedocs.io/en/stable/api/inference/serializers.html#sagemaker.serializers.NumpySerializer

In [ ]:
### numpyArrayフォーマットにシリアライズする場合

from sagemaker.serializers import NumpySerializer

input_csv = [[1,2],
             [3,4],
             [5,6]]


input_csv = [1,2,3,4,5,6]

input_csv = [['a1','a2'],
             ['b1','b2'],
             ['c1','c2']]
input_csv = np.array([1,2,3,4])

serialized = NumpySerializer().serialize(input_csv)
print(type(serialized))
print(serialized)

NumpySerializer().serialize(input_csv)

### デシリアライザについて

SageMakerSDKを使ってデプロイすると、SerializerとDeserializerはnumpy arrayが設定される。

* 予測結果もnupy array型とする。
    * LightGBMのpredict()返り値はarray型：https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMClassifier.html#lightgbm.LGBMClassifier.predict
* output_fnはnumpy_arrayのデータストリームを作る
* Deserializerで、inputはストリーム型で、numpyArray型（もしくはPandas DataFrame）で受け取る


list
https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html#sagemaker.deserializers.CSVDeserializer

Numpy ndarray
https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html#sagemaker.deserializers.NumpyDeserializer

Pandas DataFrame
https://sagemaker.readthedocs.io/en/stable/api/inference/deserializers.html#sagemaker.deserializers.PandasDeserializer


In [ ]:
CSVSerializer().serialize(pred)

In [ ]:
from sagemaker.deserializers import PandasDeserializer

#pred = [[1,2],
#             [3,4],
#             [5,6]]


pred = [1,2,3,4,5,6]
pred = 'a,b,c,d,e,f'

#pred = [['a1','a2'],
#             ['b1','b2'],
#             ['c1','c2']]

#pred = np.array([1,2,3,4])

pred = {
  "predictions": [{
    "closest_cluster": 5,
    "distance_to_cluster": 36.5
  }]
}

print(type(pred))

#deserialized = PandasDeserializer().deserialize(pred, 'application/json')
#deserialized = PandasDeserializer(accept='text/csv').deserialize(pred, 'text/csv')
#deserialized = PandasDeserializer().deserialize(pred, 'application/json')
deserialized = PandasDeserializer().deserialize(pred, 'text/csv')
#deserialized = PandasDeserializer().deserialize(CSVSerializer().serialize(pred), 'text/csv')


print(type(deserialized))
print(deserialized)

#NumpySerializer().serialize(input_csv)

botocore.response.StreamingBody()を作って試す

# デシリアライザがうまくいくケース ===================

In [ ]:
import botocore
import json
from io import BytesIO

In [ ]:
# 返却したいオブジェクト
body_json = {
    "aaa": 3,
    "bbb": [
        {
            "ccc": "ddd"
        }
    ]
}

# エンコード。(encode()はデフォルトでutf-8。)
body_encoded = json.dumps(body_json).encode()

# StreamingBodyへ整形する。
body = botocore.response.StreamingBody(BytesIO(body_encoded),len(body_encoded))

In [ ]:
#deserialized = PandasDeserializer().deserialize(body, 'text/csv')
deserialized = PandasDeserializer().deserialize(body, 'application/json') ### JSONがdeserializerのインプット


In [ ]:
print(type(deserialized))
print('='*30)
print(deserialized)
print('='*30)
deserialized

# END: デシリアライザがうまくいくケース ===================

In [ ]:
body_encoded = json.dumps(body_json).encode()

In [ ]:
print(type(body_encoded))
print(body_encoded)

In [ ]:
body

In [ ]:
import encoders

In [ ]:
pred = np.array([[1],[2],[3],[4],[5]]) ### LGBM予測の出力とする
print(type(pred))
print(pred)
pred.shape

In [ ]:
print(type(pred.tobytes()))
print(pred.tobytes())

In [ ]:
body = botocore.response.StreamingBody(BytesIO(pred.tobytes()),len(pred.tobytes()))

In [ ]:
body

In [ ]:
NumpyDeserializer().deserialize(body, 'application/x-npy') ### SageMakerSDKの場合これが使われる。
#NumpyDeserializer().deserialize(body, 'application/json')

## NumpySeriarizerからDeserializerを使う

inputは同じくndarray

In [ ]:
from sagemaker.deserializers import NumpyDeserializer

In [ ]:
pred = np.array([[1],[2],[3],[4],[5]]) ### LGBM予測の出力とする
print(type(pred))
print(pred)
pred.shape

In [ ]:
NumpySerializer().serialize(pred)

In [ ]:
NumpyDeserializer().deserialize(NumpySerializer().serialize(pred), 'application/x-npy') ### SageMakerSDKの場合これが使われる。

In [ ]:
from sagemaker.deserializers import NumpyDeserializer

In [ ]:
pred_str = pred.astype('str')

In [ ]:
pred_str

In [ ]:
# deserializer()のインプットのための、ストリームを作成
pred = np.array([1,2,3,4,5])

#encorders.encode(pred, 'utf-8')
#body = botocore.response.StreamingBody(BytesIO(pred),len(pred))
#body = botocore.response.StreamingBody(BytesIO(pred_str),len(pred_str))

body = botocore.response.StreamingBody(pred.tobytes(),len(pred.tobytes()))


#deserialized = PandasDeserializer().deserialize(body, 'application/json')
#deserialized = PandasDeserializer().deserialize(body, 'text/csv')
#deserialized = PandasDeserializer().deserialize(pred, 'application/x-npy') ### PandasDeserializer()は、CSVとJSONのみ
#deserialized = NumpyDeserializer().deserialize(body, 'application/x-npy')
deserialized = NumpyDeserializer().deserialize(body, 'text/csv')


In [ ]:
body

In [ ]:
StreamingBody(io.BytesIO(encoded_content),
                         len(encoded_content)) 

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor_lgb_model.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

# (optional) XGBoostコンテナで、LGBMの推論を実施する

LGBMのカスタムコンテナも存在する
< URL >
    

In [ ]:
xgb_container_uri = sagemaker.image_uris.retrieve("xgboost", region_name, "1.5-1")

In [ ]:
xgb_container_uri

In [ ]:
from sagemaker.predictor import RealTimePredictor

lgb_model = sagemaker.model.Model(xgb_container_uri, # XGBoostビルトインコンテナのURI
                                  model_data=est_lightgbm.model_data, # ローカル学習で生成したモデルファイル
                                  role=role,
                                  predictor_cls=RealTimePredictor, # 推論するための識別子を指定
                                  source_dir='./src_builtin_container_serve', # requirements.txt必要な場合
                                  entry_point='inference.py' # source_dirを指定している場合、.pyファイルを指定する。
                                 )

In [ ]:
!docker ps

In [ ]:
!docker stop $(docker ps -q)

In [ ]:
!docker ps

In [ ]:
predictor_lgb_model = lgb_model.deploy(initial_instance_count=1,
                                       instance_type='local', 
                                       serializer=csv_serializer, ### string形式でSageMakerに渡す（認識してもらう）
                                       #deserializer=None, 
                                      )

In [ ]:
### 推論実行
with open(local_test, 'r') as f:
    payload = f.read().strip()

predicted = predictor_lgb_model.predict(payload).decode('utf-8')
print('=' * 20)
print(predicted)

In [ ]:
print(predicted)

# END of Containts =======================

# 後片付け

# 参考

## （optional）4. カスタムコンテナを使わず、built-inコンテナのrequirement.txtにlightgbmを記載して実行する



過去バージョン（1.3-3, 1.2-2, 1.2-1, 1.0-1)はこちら

https://github.com/aws/sagemaker-xgboost-container/releases


## 4-2. 推論実施

### 4-2-1.デプロイ

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy


デプロイの際に、ソースコードを指定するにはどうしたらいいのか？

https://www.youtube.com/watch?v=sngNd79GpmE&t=596s


ポイント：あらためて、Estimatorを定義する必要がある。

### serve用のファイルは、.py かつ、作法に従う必要がある。

MMSは.pyを扱うように設計されているため。

https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator

## エラー

RuntimeError: Model /opt/ml/model/model.tar.gz cannot be loaded:


6o0805unb3-algo-1-k8ugv | [2022-10-16 02:25:50 +0000] [19] [ERROR] Exception in worker process  
6o0805unb3-algo-1-k8ugv | Traceback (most recent call last):  
6o0805unb3-algo-1-k8ugv |   File "/miniconda3/lib/python3.8/site-packages/sagemaker_xgboost_container/algorithm_mode/serve_utils.py", line 175, in get_loaded_booster  
6o0805unb3-algo-1-k8ugv |     booster = pkl.load(open(full_model_path, "rb"))  
6o0805unb3-algo-1-k8ugv | _pickle.UnpicklingError: invalid load key, '\x1f'.  

## 原因
lightgbm-regression-model.txtなので、pklでは読み込めない。

モデルロードする関数を上書きするには？？（そもそもこれがやりたい）

https://github.com/aws/sagemaker-xgboost-container/blob/master/docker/1.5-1/final/Dockerfile.cpu

# Set SageMaker entrypoints
ENV SAGEMAKER_TRAINING_MODULE sagemaker_xgboost_container.training:main  
ENV SAGEMAKER_SERVING_MODULE sagemaker_xgboost_container.serving:main  


まず、serving.main()が実行される

https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/serving.py

L143

serving_env = env.ServingEnv()

で、環境変数にパラメータが読み込まれる


L147

user_module = modules.import_module(serving_env.module_dir, serving_env.module_name)

ここで、ユーザーのモジュールが読み込まれる。

L18をみると、sagemaker_containers.beta.framework.modulesがモジュールのようだ。

from sagemaker_containers.beta.framework import (
    encoders,
    env,
    modules,
    server,
    transformer,
    worker,
)

https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/beta/framework/__init__.py

sagemaker_containers.beta.frameworkはアーカイブされている。

現在はこちら。initをみると

https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_modules.py



L258で、imortしている。

module = importlib.import_module(name)

def import_module(uri, name=DEFAULT_MODULE_NAME, cache=None):  # type: (str, str, bool) -> module

とあるように、DEFAULT_MODULE_NAMEが読み込まれるようだ





https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/serving.py

L148,149: L147で読み込んだユーザーモジュールに上書きする

user_module_transformer = _user_module_transformer(user_module)  
user_module_transformer.initialize()  


L116にあるように、model_fnなどのユーザー関数に上書きされる。


def _user_module_transformer(user_module):  
    model_fn = getattr(user_module, "model_fn", default_model_fn)  
    input_fn = getattr(user_module, "input_fn", None)  
    predict_fn = getattr(user_module, "predict_fn", None)  
    output_fn = getattr(user_module, "output_fn", None)  
    transform_fn = getattr(user_module, "transform_fn", None)  

## model_fnを定義したファイルが、importされているか？

いま、そもそも環境変数に正しく情報渡せていない気がする。


https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_modules.py

L237より、

def import_module(uri, name=DEFAULT_MODULE_NAME, cache=None):  # type: (str, str, bool) -> module

第二引数に指定する必要がある。

これを呼ぶのは、


https://github.com/aws/sagemaker-xgboost-container/blob/master/src/sagemaker_xgboost_container/serving.py

L147

user_module = modules.import_module(serving_env.module_dir, serving_env.module_name)

serving_env.module_name である。指定できているのか？


L143より

serving_env = env.ServingEnv()

これは、以下のファイル。

https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_env.py

L862

class ServingEnv(_Env):



https://github.com/aws/sagemaker-containers/blob/master/src/sagemaker_containers/_env.py

L329には、

class _Env(_mapping.MappingMixin):


module_name = os.environ.get(_params.USER_PROGRAM_ENV, None)

とある。


L595

TrainingEnvには、

        # override base class attributes  
        if self._module_name is None:  
            self._module_name = str(sagemaker_hyperparameters.get(_params.USER_PROGRAM_PARAM, None))  
        self._user_entry_point = self._user_entry_point or sagemaker_hyperparameters.get(  
            _params.USER_PROGRAM_PARAM  
        )  
        
        
        

## USER_PROGRAM_ENVに設定できればいい？


https://github.com/aws/sagemaker-inference-toolkit/blob/master/src/sagemaker_inference/parameters.py


L18

USER_PROGRAM_ENV = "SAGEMAKER_PROGRAM"  # type: str

SAGEMAKER_PROGRAMに設定できればいいようだ。

ビルトインコンテナにはどうすれば設定できるのだろうか？？

以下のYouTubeだと、boto3でEnvironment引数を使っている。

https://youtu.be/sngNd79GpmE?t=780

# デバッグのために、dockerイメージをプルして、中をみてみる。

ビルトインコンテナの中身をみるには、どうすればいいのか？

XGBoostの場合は、ローカルでbuildしていくようだ。

https://github.com/aws/sagemaker-xgboost-container

# コンテナの中に入って確認する方法
コンソールを立ち上げて、以下の流れで実行する


ディレクトリ移動
 $ cd sagemaker-xgboost-container/
 
baseコンテナをビルド
 $ docker build -t xgboost-container-base:1.5-1-cpu-py3 -f docker/1.5-1/base/Dockerfile.cpu .

finalコンテナをビルド
 $ docker build -t preprod-xgboost-container:1.5-1-cpu-py3 -f docker/1.5-1/final/Dockerfile.cpu .

構築されたイメージを確認
$ docker image ls

中に入って確認（コンテナのタグもつけて指定すること）
$ docker run -it preprod-xgboost-container:1.5-1-cpu-py3 /bin/bash

$ docker run -it 354813040037.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-xgboost:1.5-1 /bin/bash  

   

# コンテナの中に入り、 command serveを実行してみる

# モデル利用ならうまくいくのではないか？-> OK


YouTubeのリンク先ソースより

https://github.com/aws-samples/aws-ml-jp/blob/main/sagemaker/sagemaker-inference/inference-tutorial/1_sklearn.ipynb



https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model

デプロイ

https://sagemaker.readthedocs.io/en/stable/api/inference/model.html#sagemaker.model.Model.deploy

ValueError: Estimator is not associated with a training job

# エラー　trainingjobとの紐付け

https://stackoverflow.com/questions/63340328/how-to-define-a-sagemaker-estimator-object-using-a-pre-trained-model-and-then-de

# END of Containts ===============

# 5.後片付け
予期せぬ課金を防ぐために、以下のリソースを削除します。

* SageMaker 推論エンドポイント
* ECR
* S3
* SageMakerノートブックインスタンス

# 参考
* SageMaker のtrainingジョブを理解する
    * https://github.com/aws-samples/aws-ml-jp/tree/main/sagemaker/sagemaker-traning/tutorial
* SageMaker-Pytorth training Toolkit
    * https://github.com/aws/sagemaker-pytorch-training-toolkit/
* SageMaker-Pytorch Inference Toolkit
    * https://github.com/aws/sagemaker-pytorch-inference-toolkit
* SageMaker Inference Toolkit
    * https://docs.aws.amazon.com/sagemaker/latest/dg/amazon-sagemaker-toolkits.html
    * https://github.com/aws/sagemaker-inference-toolkit